In [2]:
import os
!pip install pandas


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\joshu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import os
import pandas as pd

base_dir = os.path.abspath('__file__')
loc_folders = [f"loc{i}" for i in range(1, 7)]
splits = ["train", "val"]

all_entries = []

for loc in loc_folders:
    for split in splits:
        csv_path = os.path.join(base_dir,"..", loc, f"{split}.csv")
        if not os.path.exists(csv_path):
            print(f"Missing: {csv_path}")
            continue

        df = pd.read_csv(csv_path)

        # Calculate total number of cars and CVs
        df["car_total"] = df["car_left"] + df["car_right"]
        df["cv_total"] = df["cv_left"] + df["cv_right"]

        # Assign label based on vehicle type
        def classify(row):
            if row["car_total"] > 0 and row["cv_total"] == 0:
                return 0  # only car
            elif row["cv_total"] > 0 and row["car_total"] == 0:
                return 1  # only CV
            elif row["cv_total"] == 0 and row["car_total"] == 0:
                return 2  #none
            

        df["label"] = df.apply(classify, axis=1)
        df = df[df["label"].notna()]

        # Use relative path (from base_dir)
        df["rel_path"] = df["path"].apply(lambda x: os.path.join(loc, x.replace("/", os.sep)))

        df["loc"] = loc
        df["split"] = split

        all_entries.append(df[["rel_path", "label", "loc", "split"]])

# Combine all rows
all_data = pd.concat(all_entries, ignore_index=True)

# Save to unified CSV
output_csv = os.path.join(base_dir, "..", "all_audio_labels.csv")
all_data.to_csv(output_csv, index=False)
print(f"Saved unified label file: {output_csv}")

Saved unified label file: c:\AcousticTrafficMonitoring\DataPreprocessing\__file__\..\all_audio_labels.csv


In [ ]:
!pip install librosa
!pip install tqdm
!pip install --force-reinstall numpy==2.2

import os
import pandas as pd
import librosa
import soundfile as sf
import numpy as np
from tqdm import tqdm

def normalize_energy(energy_array):
    return (energy_array - np.min(energy_array)) / (np.max(energy_array) - np.min(energy_array) + 1e-8)

def process_flac_and_save_segments(row, output_base, sr=16000,
                                   segment_duration_vehicle=1.0,
                                   segment_duration_bg=1.0,
                                   baseline_secs=3, threshold_ratio=0.3):
    #abs_path = os.path.join("D:/DATA/SD_data", row["rel_path"])  # ✅ abspath
    abs_path = os.path.join(os.path.abspath('__file__'), "..", row["rel_path"])
    label = int(row["label"])
    loc = row["loc"]
    split = row["split"]

    try:
        y, _ = librosa.load(abs_path, sr=sr)
    except Exception as e:
        print(f"❌ Error loading {abs_path}: {e}")
        return []

    entries = []

    if label == 2:
        # === Background: fixed 1s segmentation ===
        seg_len = int(sr * segment_duration_bg)
        num_segs = len(y) // seg_len

        for i in range(num_segs):
            start = i * seg_len
            end = start + seg_len
            segment = y[start:end]
            if len(segment) < seg_len:
                continue

            filename = f"{loc}_{split}_{os.path.basename(abs_path).replace('.flac','')}_bg_{i+1:03d}.wav"
            out_dir = os.path.join(output_base, "background")
            os.makedirs(out_dir, exist_ok=True)
            out_path = os.path.join(out_dir, filename)
            sf.write(out_path, segment, sr)
            entries.append({"filepath": os.path.relpath(out_path, output_base), "label": label})

    elif label in [0, 1]:
        # === Car / CV: energy threshold filtering ===
        seg_len = int(sr * segment_duration_vehicle)
        num_segs = len(y) // seg_len
        energies = [np.sqrt(np.mean(y[i * seg_len:(i + 1) * seg_len] ** 2)) for i in range(num_segs)]
        energies = np.array(energies)
        energies_norm = normalize_energy(energies)
        baseline = np.mean(energies_norm[:baseline_secs])
        threshold = baseline + threshold_ratio
        smoothed = np.convolve(energies_norm, np.ones(3) / 3, mode='same')
        active = smoothed > threshold

        for i, is_active in enumerate(active):
            if not is_active:
                continue
            start = i * seg_len
            end = start + seg_len
            segment = y[start:end]
            if len(segment) < seg_len:
                continue

            class_dir = "car" if label == 0 else "cv"
            filename = f"{loc}_{split}_{os.path.basename(abs_path).replace('.flac','')}_seg2s_{i+1:03d}.wav"
            out_dir = os.path.join(output_base, class_dir)
            os.makedirs(out_dir, exist_ok=True)
            out_path = os.path.join(out_dir, filename)
            sf.write(out_path, segment, sr)

            #save relative path
            entries.append({"filepath": os.path.join(class_dir, filename), "label": label})

    return entries

# ==== Main ====
input_csv = os.path.join(os.path.abspath('__file__'), "..", "all_audio_labels.csv")
base_dir = os.path.join(os.path.abspath('__file__'), "..")
output_base = os.path.join(base_dir, "new_vehicle_segments")
output_csv = os.path.join(output_base, "vehicle_clips.csv")

if not os.path.exists(output_base):
    os.makedirs(output_base)

df = pd.read_csv(input_csv)
df = df[df["label"].isin([0, 1, 2])]

all_entries = []
for _, row in tqdm(df.iterrows(), total=len(df)):
    entries = process_flac_and_save_segments(row, output_base)
    all_entries.extend(entries)

df_out = pd.DataFrame(all_entries)
df_out.to_csv(output_csv, index=False)
print(f"\n✅ Saved segmented vehicle clips to {output_csv}")



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\joshu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\joshu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\joshu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached numpy-2.2.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.0-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.0
    Uninstalling numpy-2.2.0:
      Successfully uninstalled numpy-2.2.0


100%|██████████| 10266/10266 [10:25<00:00, 16.42it/s]


✅ Saved segmented vehicle clips to c:\AcousticTrafficMonitoring\DataPreprocessing\__file__\..\new_vehicle_segments\vehicle_clips.csv


In [ ]:
import pandas as pd

csv_path = "D:/DATA/SD_data/vehicle_segments/vehicle_clips.csv"
df = pd.read_csv(csv_path)

car_count = (df["label"] == 0).sum()
cv_count = (df["label"] == 1).sum()
bg_count = (df["label"] == 2).sum()
print(f"Number of car segments (label=0): {car_count}")
print(f"Number of cv segments (label=1):  {cv_count}")
print(f"Number of bg segments (label=2):  {bg_count}")


Number of car segments (label=0): 71313
Number of cv segments (label=1):  1329
Number of bg segments (label=2):  11400


In [2]:
import numpy as np
import librosa
import soundfile as sf
import random
from tqdm import tqdm
import pandas as pd
import os

# Config
target_total = 71313
sr = 16000
segment_duration = 1.0
samples_per_segment = int(sr * segment_duration)

# Paths
base_dir = os.path.dirname(os.path.abspath('__file__'))
""" 
original_csv = os.path.join(base_dir, "vehicle_clips.csv")
background_dir = os.path.join(base_dir, "background")
cv_aug_dir = os.path.join(base_dir, "cv_aug") """
output_base = os.path.join(base_dir, "new_vehicle_segments", "cv")
BG_DIR = os.path.join(base_dir, "new_vehicle_segments", "background")
vehicle_csv = os.path.join(base_dir, "new_vehicle_segments", "vehicle_clips.csv")

# Load data
df = pd.read_csv(vehicle_csv)
df_cv = df[df["label"] == 1].reset_index(drop=True)
df_car = df[df["label"] == 0]

# Background pool
background_files = [os.path.join(BG_DIR, f) for f in os.listdir(BG_DIR) if f.endswith(".wav")]

# Augmentation functions
def augment_with_background(y, bg, snr_db=5):
    if len(bg) < len(y):
        bg = np.tile(bg, int(np.ceil(len(y)/len(bg))))
    bg = bg[:len(y)]
    rms_y = np.sqrt(np.mean(y**2))
    rms_bg = np.sqrt(np.mean(bg**2))
    alpha = rms_y / (10**(snr_db/20)) / (rms_bg + 1e-6)
    return y + alpha * bg

def time_stretch(y, rate):
    return librosa.effects.time_stretch(y, rate)

def pitch_shift(y, sr, n_steps):
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

def reverse(y):
    return y[::-1]

#guarentees one Augment
def applyOneAugment(aug_y):

    choice = random.randint(0,3)

    # Random augmentation combos
    if choice == 0 and background_files:
        bg_path = random.choice(background_files)
        bg_y, _ = librosa.load(bg_path, sr=sr)
        aug_y = augment_with_background(aug_y, bg_y)
    if not background_files:
        choice = random.randint(1,3)

    if choice == 1:
        if random.random() < 0.5:
            rate = random.uniform(0.8, 0.99)
        else:
            rate = random.uniform(1.01, 1.2)
        try:
            aug_y = time_stretch(aug_y, rate)
        except:
            choice += random.randint(1,2)
        

    if choice == 2:
        steps = random.randint(-2, 2)
        aug_y = pitch_shift(aug_y, sr=sr, n_steps=steps)

    if choice == 3:
        aug_y = reverse(aug_y)

    return aug_y
    
#No guarentee Augment is applied
#returns tuple, boolean and signal.
#boolean is true if signal was augmented false otherwise.
def applyManyAugments(aug_y):
    wasAugmented = False

    # Random augmentation combos
    if random.random() < 0.6 and background_files:
        bg_path = random.choice(background_files)
        bg_y, _ = librosa.load(bg_path, sr=sr)
        aug_y = augment_with_background(aug_y, bg_y)
        wasAugmented = True

    if random.random() < 0.5:
        if random.random() < 0.5:
            rate = random.uniform(0.8, 0.99)
        else:
            rate = random.uniform(1.01, 1.2)
        try:
            aug_y = time_stretch(aug_y, rate)
            wasAugmented = True
        except:
            wasAugmented = False
        

    if random.random() < 0.5:
        steps = random.randint(-2, 2)
        aug_y = pitch_shift(aug_y, sr=sr, n_steps=steps)
        wasAugmented = True

    if random.random() < 0.3:
        aug_y = reverse(aug_y)
        wasAugmented = True

    return wasAugmented, aug_y

# Start augmentation loop
aug_entries = []
existing = len(df_cv)
needed = target_total - existing
cv_paths = df_cv["filepath"].tolist()
error = False

with tqdm(total = needed) as pbar:
    while len(aug_entries) < needed and not error:
        for path in cv_paths:
            if len(aug_entries) >= needed:
                break
            try:
                y, _ = librosa.load(os.path.join(base_dir, "new_vehicle_segments", path), sr=sr)
            except:
                print(f"ERROR failed to load {path}!")
                error = True
                break

            y_copy = y.copy()

            augmented, aug_y = applyManyAugments(y_copy)

            #if no augment applied do something
            if not augmented:
                aug_y = applyOneAugment(y_copy)

            # Save augmented segment
            base_name = os.path.splitext(os.path.basename(path))[0]
            aug_name = f"{base_name}_aug{len(aug_entries)}.wav"
            aug_path = os.path.join(output_base, aug_name)
            sf.write(aug_path, aug_y, sr)

            aug_entries.append({
                "filepath": aug_path.replace("\\", "/"),
                "label": 1
            })
            pbar.update(1)

if not error:
    # Save final CSV
    df_aug = pd.DataFrame(aug_entries)
    df_final = pd.concat([df, df_aug], ignore_index=True)
    df_final.to_csv(vehicle_csv, index=False)
    print(f"✅ Augmentation completed! Generated {len(df_aug)} CV samples and saved to vehicle_clips.csv")
else:
    print("⛔ Aborting!")

100%|██████████| 69984/69984 [06:59<00:00, 166.77it/s]


✅ Augmentation completed! Generated 69984 CV samples and saved to vehicle_clips.csv
